In [1]:
import os
import pandas as pd
from IPython.display import display

In [2]:
print(os.getcwd())

c:\Users\YooNi\OneDrive\Desktop\Majorana-Neutrino-Hunt\extracted_features_csv_files


In [3]:
ls

 Volume in drive C is OS
 Volume Serial Number is 56AE-130D

 Directory of c:\Users\YooNi\OneDrive\Desktop\Majorana-Neutrino-Hunt\extracted_features_csv_files

01/25/2026  02:52 AM    <DIR>          .
01/21/2026  08:13 PM    <DIR>          ..
01/25/2026  02:53 AM            82,818 combine_everyone.ipynb
01/24/2026  11:49 PM    <DIR>          combined_csv_files
01/21/2026  08:12 PM               311 description.txt
01/25/2026  02:52 AM    <DIR>          eunice_csv_files
01/21/2026  08:12 PM    <DIR>          jade_csv_files
01/21/2026  08:12 PM        23,129,082 labels_test.csv
01/21/2026  08:12 PM        61,998,207 labels_train.csv
01/21/2026  08:12 PM    <DIR>          nomin_csv_files
01/24/2026  11:49 PM    <DIR>          prithvi_csv_files
               4 File(s)     85,210,418 bytes
               7 Dir(s)  105,595,662,336 bytes free


In [4]:
def process_and_merge_features(
    labels_path: str,
    eunice_path: str,
    nomin_path: str,
    prithvi_path: str,
    # jade_path: str,
    output_filename: str,
    output_dir: str = "combined_csv_files",
) -> pd.DataFrame:
    """
    Loads feature CSVs from different sources, applies specific preprocessing 
    (specifically to Jade's IDs), merges them with labels, and saves the result.
    """
    
    # Load Data
    print("=" * 50)
    print("Loading Files...")
    print("=" * 50)
    
    try:
        labels = pd.read_csv(labels_path)
        e = pd.read_csv(eunice_path)
        n = pd.read_csv(nomin_path)
        p = pd.read_csv(prithvi_path)
        # j = pd.read_csv(jade_path)
    except FileNotFoundError as err:
        print(f"Error loading files: {err}")
        return None

    # Fix Jade IDs (Specific Logic)
    # print("Preprocessing Jade IDs...")
    # g = j["id"].str.replace("id_train_", "", regex=False).astype(int)
    # file_idx = (g // 65000).astype(int)
    # j["id"] = g.astype(str) + "_train_" + file_idx.astype(str)

    # Preview Data    
    data_map = {
        "Labels": labels,
        "Eunice": e,
        "Nomin": n,
        "Prithvi": p,
        # "Jade": j
    }

    for name, df in data_map.items():
        print(f"\n{name} — File Shape: {df.shape}")
        display(df.head())

    # Merge
    print("\n" + "=" * 50)
    print("Merging Datasets...")
    
    merged = (
        labels
        .merge(e, on="id", how="inner")
        .merge(n, on="id", how="inner")
        .merge(p, on="id", how="inner")
        # .merge(j, on="id", how="inner")
    )

    print(f"Final Merged Shape: {merged.shape}")

    # Save Output
    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, output_filename)
    
    print(f"Saving to {out_path}...")
    merged.to_csv(out_path, index=False, compression="gzip")
    print("Save Complete.")

    return merged

## Combine Training Files

In [5]:
paths_train = {
    "labels_path": "labels_train.csv",
    "eunice_path": "eunice_csv_files/train_csv/train_all_features.csv.gz",
    "nomin_path": "nomin_csv_files/combined_train_n.csv.gz",
    "prithvi_path": "prithvi_csv_files/train_2.csv.gz",
    # "jade_path": "jade_csv_files/train_jade_features.csv"
}

merged_df_train = process_and_merge_features(
    **paths_train,
    output_filename="combined_train_with_labels.csv.gz")

merged_df_train.head()

Loading Files...



Labels — File Shape: (1040000, 7)


,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0
0,0_train_0,582.364295,False,True,True,True,957
1,1_train_0,250.159995,False,True,True,True,948
2,2_train_0,1212.323954,False,True,False,True,965
3,3_train_0,240.878110,False,True,True,False,927
4,4_train_0,285.124189,False,True,True,False,958



Eunice — File Shape: (1040000, 7)


,id,ED,HWP,LQ80,PPR,SCA,ND80
0,0_train_0,3409.0,2120.0,-635333.796311,0.719376,0.034655,0.0
1,1_train_0,3404.0,2004.0,-289756.893085,0.729709,0.035314,0.0
2,2_train_0,3411.0,2125.0,-379843.029134,0.715390,0.034915,0.0
3,3_train_0,3408.0,2098.0,-252673.626844,0.769375,0.034752,0.0
4,4_train_0,3406.0,2037.0,-317761.453582,0.728165,0.035132,0.0


...

,id,current_skewness,spectral_centroid_power,tail_charge_diff,current_kurtosis,total_power,time_to_main_peak,time_to_peak,late_over_early
0,0_train_0,1.780811,107.276207,-0.598625,2.117825,1.709302e+09,85,85,0.987910
1,1_train_0,1.756635,108.213621,-0.573984,2.058622,2.991376e+08,87,87,0.988301
2,2_train_0,1.079789,105.735183,-0.620023,-0.067003,6.244385e+08,95,95,0.987491
3,3_train_0,1.899438,107.946935,-0.571186,2.443885,2.811583e+08,116,116,0.988450
4,4_train_0,1.156442,106.350372,-0.563259,-0.173890,3.885442e+08,94,94,0.988541



Prithvi — File Shape: (1040000, 8)


,id,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
0,0_train_0,32.0,65.0,85.0,0.142357,3,1.198436,0.059642
1,1_train_0,0.0,60.0,87.0,0.151160,3,1.381123,0.061975
2,2_train_0,35.0,67.0,95.0,0.142606,3,1.361857,0.050813
3,3_train_0,0.0,90.0,116.0,0.133192,2,1.165654,0.062954
4,4_train_0,0.0,61.0,94.0,0.145504,7,1.390918,0.055013



Merging Datasets...
Final Merged Shape: (1040000, 28)
Saving to combined_csv_files\combined_train_with_labels.csv.gz...
Save Complete.


,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,time_to_main_peak,time_to_peak,late_over_early,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
0,0_train_0,582.364295,False,True,True,True,957,3409.0,2120.0,-635333.796311,...,85,85,0.987910,32.0,65.0,85.0,0.142357,3,1.198436,0.059642
1,1_train_0,250.159995,False,True,True,True,948,3404.0,2004.0,-289756.893085,...,87,87,0.988301,0.0,60.0,87.0,0.151160,3,1.381123,0.061975
2,2_train_0,1212.323954,False,True,False,True,965,3411.0,2125.0,-379843.029134,...,95,95,0.987491,35.0,67.0,95.0,0.142606,3,1.361857,0.050813
3,3_train_0,240.878110,False,True,True,False,927,3408.0,2098.0,-252673.626844,...,116,116,0.988450,0.0,90.0,116.0,0.133192,2,1.165654,0.062954
4,4_train_0,285.124189,False,True,True,False,958,3406.0,2037.0,-317761.453582,...,94,94,0.988541,0.0,61.0,94.0,0.145504,7,1.390918,0.055013


In [6]:
df_train = pd.read_csv('combined_csv_files/combined_train_with_labels.csv.gz')
df_train

,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,time_to_main_peak,time_to_peak,late_over_early,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
0,0_train_0,582.364295,False,True,True,True,957,3409.0,2120.0,-635333.796311,...,85,85,0.987910,32.0,65.0,85.0,0.142357,3,1.198436,0.059642
1,1_train_0,250.159995,False,True,True,True,948,3404.0,2004.0,-289756.893085,...,87,87,0.988301,0.0,60.0,87.0,0.151160,3,1.381123,0.061975
2,2_train_0,1212.323954,False,True,False,True,965,3411.0,2125.0,-379843.029134,...,95,95,0.987491,35.0,67.0,95.0,0.142606,3,1.361857,0.050813
3,3_train_0,240.878110,False,True,True,False,927,3408.0,2098.0,-252673.626844,...,116,116,0.988450,0.0,90.0,116.0,0.133192,2,1.165654,0.062954
4,4_train_0,285.124189,False,True,True,False,958,3406.0,2037.0,-317761.453582,...,94,94,0.988541,0.0,61.0,94.0,0.145504,7,1.390918,0.055013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039995,1039995_train_15,210.834626,True,True,True,True,949,3403.0,2048.0,-240759.341995,...,90,90,0.985736,0.0,55.0,90.0,0.151747,6,1.008092,0.062825
1039996,1039996_train_15,380.170340,True,True,True,True,949,3404.0,2045.0,-417622.302428,...,82,82,0.987814,0.0,57.0,79.0,0.150305,3,1.245492,0.062706
1039997,1039997_train_15,370.109563,False,True,True,True,947,3409.0,2063.0,-407730.168602,...,102,102,0.987833,0.0,77.0,102.0,0.142950,2,1.149456,0.062153
1039998,1039998_train_15,98.258524,True,True,True,False,921,3400.0,2001.0,-116570.568893,...,92,92,0.989948,0.0,71.0,92.0,0.153135,5,1.192639,0.065264


In [7]:
df_train.describe()

,energy_label,tp0,ED,HWP,LQ80,PPR,SCA,ND80,current_skewness,spectral_centroid_power,...,time_to_main_peak,time_to_peak,late_over_early,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
count,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.039995e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,...,1.040000e+06,1.040000e+06,1.040000e+06,1.039995e+06,1.039995e+06,1.039995e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06
mean,6.381633e+02,9.516434e+02,3.406036e+03,2.054120e+03,-6.978950e+05,7.254552e-01,3.572992e-02,3.009126e-03,1.906792e+00,1.145299e+02,...,1.082445e+02,1.144218e+02,9.893650e-01,1.987803e+01,6.981389e+01,1.131437e+02,1.495163e-01,5.104236e+00,1.777303e+00,6.049704e-02
std,6.704271e+02,1.895332e+01,7.735526e+00,1.296720e+02,7.301064e+05,2.832694e-02,6.330801e-03,4.127131e-02,5.312883e-01,1.069477e+02,...,3.486590e+01,7.294667e+01,1.121635e-01,2.497331e+01,2.934827e+01,7.287391e+01,3.987192e-02,7.352031e+00,1.174284e+00,2.152323e-02
min,0.000000e+00,7.100000e+01,3.074000e+03,7.200000e+01,-5.846630e+06,-7.703542e+00,2.676319e-02,0.000000e+00,-1.809541e+00,8.986718e+01,...,1.000000e+00,-8.040000e+02,1.302615e-01,0.000000e+00,0.000000e+00,9.000000e+00,1.968922e-02,1.000000e+00,6.992497e-01,3.636530e-02
25%,2.383612e+02,9.420000e+02,3.404000e+03,2.032000e+03,-6.984126e+05,7.088948e-01,3.452545e-02,0.000000e+00,1.584701e+00,1.067439e+02,...,8.700000e+01,8.700000e+01,9.873562e-01,0.000000e+00,5.400000e+01,8.600000e+01,1.408390e-01,1.000000e+00,1.116426e+00,5.533778e-02
50%,3.411866e+02,9.540000e+02,3.407000e+03,2.066000e+03,-3.769554e+05,7.211736e-01,3.480813e-02,0.000000e+00,1.939160e+00,1.086075e+02,...,1.030000e+02,1.040000e+02,9.879051e-01,8.000000e+00,6.900000e+01,1.020000e+02,1.457415e-01,2.000000e+00,1.321930e+00,6.133205e-02
75%,6.300128e+02,9.640000e+02,3.410000e+03,2.099000e+03,-2.654720e+05,7.361495e-01,3.524030e-02,0.000000e+00,2.253204e+00,1.103433e+02,...,1.220000e+02,1.230000e+02,9.885530e-01,3.600000e+01,8.300000e+01,1.210000e+02,1.509235e-01,5.000000e+00,1.908089e+00,6.423424e-02
max,4.998508e+03,1.352000e+03,3.790000e+03,3.796000e+03,3.675492e+07,9.768917e-01,2.028693e-01,2.193201e+00,5.229496e+00,1.637442e+04,...,4.960000e+02,2.919000e+03,4.640625e+01,2.666000e+03,2.839000e+03,2.919000e+03,3.990051e+00,5.500000e+01,3.667323e+01,2.569411e+00


## Combine Test Files

In [8]:
paths_test = {
    "labels_path": "labels_test.csv",
    "eunice_path": "eunice_csv_files/test_csv/test_all_features.csv.gz",
    "nomin_path": "nomin_csv_files/combined_test_n.csv.gz",
    "prithvi_path": "prithvi_csv_files/test_2.csv.gz",
    # "jade_path": "jade_csv_files/test_jade_features.csv"
}

merged_df_test = process_and_merge_features(
    **paths_test,
    output_filename="combined_test_with_labels.csv.gz")

merged_df_test.head()

Loading Files...
Error loading files: [Errno 2] No such file or directory: 'eunice_csv_files/test_csv/test_all_features.csv.gz'


AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
df_test = pd.read_csv('combined_csv_files/combined_test_with_labels.csv.gz')
df_test

,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,spectral_centroid_power,tail_charge_diff,late_over_early,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr


In [ ]:
df_test.describe()

,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,spectral_centroid_power,tail_charge_diff,late_over_early,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
